In [ ]:
import numpy as np
import pandas as pd
import os
import skimage
import seaborn as sns
import tensorflow as tf

datasetsPath = "./datasets"

print("Path to dataset files:", datasetsPath)

labels = ['pneumonia', 'normal']
image_size = 224

In [ ]:
def get_data (dir_relative_path):
    data = []
    for label in labels:
        currentPath = os.path.join(datasetsPath, dir_relative_path, label)
        print("Working inside folder: ", currentPath)
        class_number = labels.index(label)

        for image in os.listdir(currentPath):
            try:
                if image.endswith('.DS_Store'):
                    continue

                image_path = os.path.join(currentPath, image)
                image_file = tf.io.read_file(image_path)
                
                image_arr = tf.image.decode_image(image_file, channels=1)
                                
                resized_arr = tf.image.resize(image_arr, [image_size, image_size])

                data.append([resized_arr, class_number])
            except Exception as ex:
                print(ex)

    return np.array(data, dtype="object")

# Data Augmentation
Import the necessary packages and resize the images

In [ ]:
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D,Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1. / 255, 
                                    horizontal_flip = True,
                                    vertical_flip = False,
                                    rotation_range = 20, # randomly rotate the image up to 20 degrees
                                    shear_range = 0.1, # skew the sides of the images by up to 10 degrees 
                                    width_shift_range = 0.2, # shift the images left/right
                                    height_shift_range = 0.2, # shift the images up/down TBD: needs more testing of values
                                    brightness_range = [0.8, 1.2],
                                    fill_mode = "nearest")

valid_datagen = ImageDataGenerator(rescale= 1. / 255)
test_datagen = ImageDataGenerator(rescale= 1. / 255)



In [ ]:
train_generator = train_datagen.flow_from_directory("./datasets/train",
                                  batch_size = 64,
                                  target_size=(image_size, image_size),
                                  class_mode = "binary",
                                  shuffle=True,
                                  seed = 42,
                                  color_mode= "grayscale")

test_generator = test_datagen.flow_from_directory("./datasets/test",
                                  batch_size = 64,
                                  target_size=(image_size, image_size),
                                  class_mode = "binary",
                                  shuffle=True,
                                  seed = 42,
                                  color_mode= "grayscale")

valid_generator = valid_datagen.flow_from_directory("./datasets/val",
                                  batch_size = 64,
                                  target_size=(image_size, image_size),
                                  class_mode = "binary",
                                  shuffle=True,
                                  seed = 42,
                                  color_mode= "grayscale")

In [ ]:
class_labels = train_generator.class_indices

print(class_labels)

# Flip them around so we have "index : value"
class_name = {}

for value, index in class_labels.items():
    class_name[index] = value

print(class_name)

In [ ]:
model = Sequential()

# Convolutional layer with minimal filters
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(image_size, image_size, 1)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())  # Add batch normalization
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Fully connected block
model.add(Flatten())
#model.add(Dense(64, activation='relu'))  # Additional dense layer
#model.add(Dropout(0.5))  # Reduce overfitting
#model.add(Dense(1, activation='sigmoid'))

model.add(Dense(256, activation= 'relu')),
model.add(Dropout(0.3))
model.add(Dense(512, activation= 'relu')),
model.add(Dropout(0.1))
model.add(Dense(1, activation= 'sigmoid'))


optimizer = Adam(learning_rate=0.0005)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
early_stop = EarlyStopping(monitor= "val_loss",
                   verbose= 1,
                   mode= "min",
                   patience= 15)


rate_reduction = ReduceLROnPlateau(monitor="val_accuracy",
                        patience= 3,
                        verbose= 1,
                        factor= 0.8, # "learning rate * factor" after patience runs out
                        min_lr= 0.0005)

In [ ]:
history = model.fit(train_generator, 
            steps_per_epoch=82,
            epochs=30,
            callbacks=[early_stop, rate_reduction],
            validation_data=valid_generator)



val_eval = model.evaluate(valid_generator)
test_eval = model.evaluate(test_generator)

In [ ]:
print(f"Validation Loss: {val_eval[0]}")
print(f"Validation Accuarcy: {val_eval[1]}")
print(f"Test Loss: {test_eval[0]}")
print(f"Test Accuarcy: {test_eval[1]}")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()